# Advanced Quantum Algorithms in Practice with Classiq

When we work on classical algorithm development, let's say in Python, we use high-level languages. This code can be translated to low-level languages, for example, assembly.  

In quantum computing, when we work on quantum algorithm development, most of the time we see the circuit implementation. This is exactly like assembly. A quantum circuit is the lowest level that we implement on the quantum computer (this information is mentioned in the video; I think working directly with pulses is even lower-level). On the other hand, we have the design of a quantum algorithm (high-level languages), and that's what we really want to do.

## What, Why, and How Classiq?

Classiq is a software platform that enables the design, optimization, analysis, and execution of quantum algorithms by utilizing the high-level functional design paradigm.

There are two interfaces. There's an IDE, a web-based interface [platform.classiq.io](https://platform.classiq.io/) (honestly, the page is quite good), and in addition, they have a Python SDK package (`from classiq import *`).


## Desing of novel Quantum Algorithms with Classiq

In [ ]:
#First, install dependeces

#!pip install classiq

Para preparar un estado en Classiq, se puede resumir en los siguientes paso:

- Definir el entorno de la funcón usando el decorador `@qfunc`
- Definir la función cuantica que preparará el estado
- Asignar los Qubits que se utilizarán, esto usando `allocate`
- Aplicar las operaciones cuanticas

Por ejemplo el estado $|-\rangle$ se prepara usando la siguiente función

In [17]:
from classiq import H, Output, QBit, X, allocate, qfunc, authenticate, CX

#Esta funcion prepara el estado |->:

@qfunc #decorador que indica que esta función es cuántica
def prepare_minus(target: Output[QBit]): 
    allocate(out=target, num_qubits=1) #Reserva un qubit en el circuito y lo asigna al destino (target)
    X(target) # Aplica la compuerta X al estado en target
    H(target) # Aplica la compuerta H al qubit en target

Para ver el estado debemos definir una función `main` la cual será luego usada para crear el modelo a ejecutar usando la función `create_model` y luego sinthetizarla usando `synthesize`. Pero primero debemos aunteticarnos

In [2]:
authenticate(overwrite=True)

c:\Users\Lenovo\OneDrive\Desktop\computacion cuantica\Proyecto_Classiq\Classiq\Lib\site-packages\classiq\_internals\authentication\token_manager.py:93: UserWarning: Overwriting an existing refresh token should only be done if it is compromised. Make sure this operation is necessary, and if not, remove the call to device registration.
  warnings.warn(


Your user code: CPFM-MQDN
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=CPFM-MQDN


In [3]:
from classiq import create_model, hadamard_transform, show, synthesize, QNum

@qfunc
def main(x: Output[QNum]):
    prepare_minus(x)


qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/497f0485-7e65-49b6-8afb-869b4cee4272?version=0.58.1


Ahora prepararé el estado GHZ = $\frac{1}{\sqrt 2}(|000\rangle + |111\rangle)$

In [12]:
from classiq import QArray, repeat, execute


@qfunc
def main(reg: Output[QArray]):
    allocate(3, reg)
    H(reg[0])
    repeat(
        count=reg.len - 1,
        iteration=lambda index: CX(ctrl=reg[index], target=reg[index + 1]),
    )


qprog = synthesize(create_model(main))

job = execute(qprog)
results = job.result()[0].value.parsed_counts
display(results)

#Para ver el resultado en la pagina oficial 

show(qprog)

[{'reg': [0, 0, 0]}: 1057, {'reg': [1, 1, 1]}: 991]

Opening: https://platform.classiq.io/circuit/99f9cf59-d9ff-4f6b-8d47-1ee486214553?version=0.58.1


# Suzuki-Trotter Approximation in Quantum Computing

## Context
The **Suzuki-Trotter approximation** is a method used to simulate the time evolution of a quantum system governed by a Hamiltonian $ H $. The time evolution is described by:
$$
U(t) = e^{-i H t},
$$
where:
- $ H $ is the Hamiltonian.
- $ t $ is the evolution time.

If the Hamiltonian can be decomposed into multiple non-commuting terms:
$$
H = H_1 + H_2 + \dots + H_k,
$$
direct computation of $ e^{-i H t} $ becomes challenging because $ e^{-i (H_1 + H_2) t} \neq e^{-i H_1 t} e^{-i H_2 t} $ when $ [H_1, H_2] \neq 0 $.

---

## Suzuki-Trotter Decomposition

### First-Order Approximation
The evolution operator can be approximated as:
$$
e^{-i H t} \approx \left( e^{-i H_1 \Delta t} e^{-i H_2 \Delta t} \right)^N,
$$
where:
- $ \Delta t = \frac{t}{N} $ is a small time step.
- $ N $ is the number of steps.

### Second-Order Approximation
To improve accuracy, a higher-order approximation is used:
$$
e^{-i H t} \approx \left( e^{-i H_1 \Delta t / 2} e^{-i H_2 \Delta t} e^{-i H_1 \Delta t / 2} \right)^N.
$$

---

## Application in Quantum Computing

### Steps:
1. **Decompose the Hamiltonian**:
   Identify the terms $ H_1, H_2, \dots, H_k $ in the Hamiltonian.

2. **Implement $ e^{-i H_i \Delta t} $**:
   Each term $ e^{-i H_i \Delta t} $ is implemented as a series of quantum gates.

3. **Iterate**:
   Use the Suzuki-Trotter approximation to construct $ e^{-i H t} $ from the smaller steps $ \Delta t $.

---

## Example: Spin Chain Hamiltonian

For a Hamiltonian:
$$
H = J \sum_{i} Z_i Z_{i+1} + h \sum_{i} X_i,
$$
we decompose:
$$
H_1 = J \sum_{i} Z_i Z_{i+1}, \quad H_2 = h \sum_{i} X_i.
$$
Using the first-order Suzuki-Trotter approximation:
$$
e^{-i H t} \approx \left( e^{-i H_1 \Delta t} e^{-i H_2 \Delta t} \right)^N.
$$

---

## Advantages and Limitations

### Advantages:
- **Flexible**: Allows simulation of complex systems using simple gates.
- **Scalable**: Increasing $ N $ improves the accuracy.

### Limitations:
- **Truncation Errors**: Errors depend on $ \Delta t $ and the order of the approximation.
- **Cost**: Higher precision requires more quantum gates.

---

## Summary

The Suzuki-Trotter approximation is a foundational tool for simulating Hamiltonian dynamics in quantum computing, enabling decomposition of complex systems into manageable components. However, it requires careful trade-offs between precision and computational cost.


In [ ]:
from classiq import *


@qfunc
def main(a: CReal, x: CReal, qba: Output[QArray[QBit]]):
    allocate(3, qba)
    suzuki_trotter(
        [
            PauliTerm(pauli=[Pauli.X, Pauli.X, Pauli.Z], coefficient=a),
            PauliTerm(pauli=[Pauli.Y, Pauli.X, Pauli.Z], coefficient=0.5),
        ],
        evolution_coefficient=x, #controla el tiempo  de evoluvión
        order=1, #El porden de la aproxx
        repetitions=1, # corresponde al N 
        qbv=qba, # registro cuatico de qbits
    )


qmod = create_model(main, out_file="suzuki_trotter")
qprog = synthesize(qmod)
#show(qprog)

En el caso anterior, el código está aproximando el siguiente hamiltoniano:

$$
H = a\cdot(X\otimes X\otimes Z)+ 0.5\cdot(Y\otimes X \otimes Z)
$$

Y su aproximación es la siguiente:

$$
U(t)\approx e^{-iat((X\otimes X\otimes Z))} + e^{-i(0.5t)(Y\otimes X \otimes Z)}
$$
